In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
ON_CPU = device == torch.device('cpu')

In [ ]:
DIR_INPUT = '/kaggle/input/smartathon-dataset-zip/dataset'
train = pd.read_csv(f'{DIR_INPUT}/train.csv')
train.shape

In [ ]:
train['xmax'] = train['xmax'].apply(lambda x : 0 if x < 0 else x*2)
train['ymax'] = train['ymax'].apply(lambda x : 0 if x < 0 else x*2)
train['xmin'] = train['xmin'].apply(lambda x : 0 if x < 0 else x*2)
train['ymin'] = train['ymin'].apply(lambda x : 0 if x < 0 else x*2)
train['xmax'] = train['xmax'].apply(lambda x : x if x < 1920 else 1920)
train['ymax'] = train['ymax'].apply(lambda x : x if x < 1080 else 1080)
train['xmax'] = train['xmax'].apply(lambda x : x/2)
train['ymax'] = train['ymax'].apply(lambda x : x/2)
train['xmin'] = train['xmin'].apply(lambda x : x/2)
train['ymin'] = train['ymin'].apply(lambda x : x/2)
train['class'] = train['class'].astype(int)

In [ ]:
image_ids = train['image_path'].unique()

train_ids = image_ids[:]

train_df = train[train['image_path'].isin(train_ids)].copy()

In [ ]:
class CreateDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_path'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_path'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        area = torch.as_tensor(area, dtype=torch.float32)

        labels = records['class']
        labels = labels.to_numpy(dtype= np.int64)
        labels = torch.from_numpy(labels)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = torch.from_numpy(boxes)
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
# Albumentations
def get_train_transform():
    return A.Compose([
        A.Resize(960, 540),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        A.Resize(960, 540),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
def get_model():
    """
    https://stackoverflow.com/questions/58362892/resnet-18-as-backbone-in-faster-r-cnn
    """
    backbone = resnet_fpn_backbone('resnet101', pretrained=True)
    model = FasterRCNN(backbone, num_classes=11)
    return model

In [ ]:
#model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
CLASSES = train['class'].unique()
CLASSES = list(CLASSES)
#CLASSES.append('__background__')
#CLASSES
num_classes = len(CLASSES)

In [ ]:
# get number of input features for the classifier
# in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

DIR_TRAIN = '/kaggle/input/smartathon-dataset-zip/dataset/images'
train_dataset = CreateDataset(train_df, DIR_TRAIN,get_train_transform())

indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

In [ ]:
model = get_model()
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
lr_scheduler = None

num_epochs = 23

In [ ]:
loss_hist = Averager()
itr = 1

model.train()

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in train_data_loader:
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 500 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")
    
    if epoch%2 == 1:
        torch.save(model.state_dict(), f'/kaggle/working/fastrcnn{epoch}.pt')